In [1]:
import numpy as np
import pandas as pd
import re
from sklearn.linear_model import LogisticRegression
import sklearn.feature_extraction.text as sktext
from sklearn.model_selection import train_test_split, cross_val_predict, cross_val_score
from sklearn import metrics
from sklearn.pipeline import make_pipeline
import warnings

In [2]:
warnings.filterwarnings('ignore')

In [3]:
restored = pd.read_json('cleaned_data.json.gz')
restored.head(5)

,class,review_markup
0,neutral,"Холодильником користуюсь близько трьох тижнів,..."
1,positive,Любителям класики пораджу придбання двокамерно...
2,positive,Місяць вже як користуюся холодильником Купувал...
3,positive,Все чудово працює..стильний і гарна ціна!
4,positive,Питання до користувачів цим холодильником або ...


In [4]:
X_train, X_test, y_train, y_test = train_test_split(restored['review_markup'], restored['class'], test_size=0.25, random_state=42, stratify=restored['class'])

In [5]:
X_train.name = 'train'
X_test.name = 'test'

In [6]:
def preprocess(text):
    return text
text_pipe_logit = make_pipeline(
    sktext.CountVectorizer(preprocessor=preprocess, analyzer='char', ngram_range=(1, 4)),
    LogisticRegression(n_jobs=-1, random_state=42)
)

In [7]:
def evaluate(label, predictor, X, y):
    print("{}. Evaluating on {}.".format(label, X.name))
    y_pred = predictor.predict(X)
    accuracy = metrics.accuracy_score(y, y_pred)
    print(metrics.confusion_matrix(y, y_pred, labels=['positive', 'negative', 'neutral']))
    print("Accuracy: {}".format(accuracy))
    return accuracy

In [8]:
def avg(numbers):
    return float(sum(numbers)) / max(len(numbers), 1)

def cross_validate(label, predictor, X, y):
    scores = cross_val_score(predictor, X, y, scoring='accuracy', cv=5, n_jobs=-1)
    print(avg(scores), '<-', scores)
    y_pred = cross_val_predict(text_pipe_logit, X, y, cv=5, n_jobs=-1)
    return metrics.confusion_matrix(y, y_pred, labels=('positive', 'neutral', 'negative'))

In [9]:
def preprocess(text):
    return text
text_pipe_logit = make_pipeline(
    sktext.CountVectorizer(preprocessor=preprocess, analyzer='char', ngram_range=(1, 4)),
    LogisticRegression(n_jobs=-1, random_state=42)
)

In [42]:
label  = "Vanilla LogReg w/ CountVectorizer on char n-grams 1-4"
cross_validate(label, text_pipe_logit, X_train, y_train)

0.899319364172223 <- [0.8960396  0.8960396  0.90049751 0.89949749 0.90452261]


array([[858,   4,  11],
       [ 32,   9,   2],
       [ 51,   1,  35]])

чи допоможе class_weight='balanced' ?

In [49]:
def preprocess(text):
    return text
text_pipe_logit = make_pipeline(
    sktext.CountVectorizer(preprocessor=preprocess, analyzer='char', ngram_range=(1, 4)),
    LogisticRegression(n_jobs=-1, class_weight='balanced', random_state=42)
)

In [50]:
label  = "LogReg class_weight=balanced w/ CountVectorizer on char n-grams 1-4"
cross_validate(label, text_pipe_logit, X_train, y_train)

0.8983441912780492 <- [0.88613861 0.8960396  0.90049751 0.89949749 0.90954774]


array([[854,   6,  13],
       [ 31,   9,   3],
       [ 47,   2,  38]])

схоже, що ні. Спробуємо використати словник тональності

In [9]:
uk_vocab = pd.read_csv('https://raw.githubusercontent.com/lang-uk/tone-dict-uk/master/tone-dict-uk-auto.tsv', names=['word', 'score'], header=None, sep='\t')
uk_vocab.head(5)

,word,score
0,Всевишній,0.885078
1,Господь,0.816095
2,Христовий,0.814423
3,аборт,0.050470
4,аварія,0.036755


In [45]:
def preprocess(text):
    return text
text_pipe_logit = make_pipeline(
    sktext.CountVectorizer(preprocessor=preprocess, analyzer='word', vocabulary=uk_vocab['word'].ravel()),
    LogisticRegression(n_jobs=-1, random_state=42)
)

In [46]:
label  = "LogReg w/ CountVectorizer on words based on sentiment vocab"
cross_validate(label, text_pipe_logit, X_train, y_train)

0.8704127504177703 <- [0.86633663 0.86633663 0.87064677 0.87437186 0.87437186]


array([[873,   0,   0],
       [ 43,   0,   0],
       [ 87,   0,   0]])

Якість впала порівняно с буквеними н-грамами, при цьому модель по суті видавала єдиний клас. Перевіримо, наскільки словник перетинається з словами з датасету

In [71]:
vocab = sktext.CountVectorizer(preprocessor=preprocess, analyzer='word', vocabulary=uk_vocab['word'].ravel())
senti_words = vocab.transform(X_train)
uk_vocab[(senti_words.sum(axis=0) > 0).T]

,word,score
10,акуратний,0.789571
20,багатофункціональність,0.778706
71,бомба,0.019866
74,брак,0.044983
93,важкий,0.781705
109,веселий,0.869653
110,вечір,0.825002
144,вирізати,0.044759
147,високий,0.870438
156,витягнути,0.032205


лише 86 слів перетнулися, що призвело до дуже sparse матриці та напевно багатьох рядків, що повністю складаються з нулів.

In [67]:
senti_words[senti_words.sum(axis=1) == 0].shape

(1, 700)

маємо таких рядків більше половини. Можливо, лематизація могла би допомогти.

In [10]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer(lang='uk')

In [33]:
def words2lemmas (str):
    words = re.split("\s", str)
    res = []
    for word in words:
        tag = morph.parse(word)
        res.append(tag[0].normal_form)
    return " ".join(res)

In [12]:
X_lemmas = X_train.apply(words2lemmas)
vocab = sktext.CountVectorizer(preprocessor=preprocess, analyzer='word', vocabulary=uk_vocab['word'].ravel())
senti_words = vocab.transform(X_lemmas)
uk_vocab[(senti_words.sum(axis=0) > 0).T]

,word,score
7,адаптований,0.797213
10,акуратний,0.789571
18,багатий,0.849303
20,багатофункціональність,0.778706
71,бомба,0.019866
73,боятися,0.020242
74,брак,0.044983
93,важкий,0.781705
94,валити,0.014925
109,веселий,0.869653


Стало краще, кілкість слів збільшилась вдвічі. Перевіримо, як це вплинуло на якість підходу з використанням словника

In [20]:
def preprocess(text):
    return text
text_pipe_logit = make_pipeline(
    sktext.CountVectorizer(preprocessor=preprocess, analyzer='word', vocabulary=uk_vocab['word'].ravel()),
    LogisticRegression(n_jobs=-1, random_state=42)
)

In [24]:
label  = "LogReg w/ CountVectorizer on words' lemmas based on sentiment vocab"
cross_validate(label, text_pipe_logit, X_lemmas, y_train)

0.8724228006690267 <- [0.86633663 0.86633663 0.87064677 0.87939698 0.87939698]


array([[873,   0,   0],
       [ 43,   0,   0],
       [ 85,   0,   2]])

Якість покращилися, але дуже незначним чином. Отримавши речення з лише леммами, скористаємося двома початковими підходами

In [26]:
def preprocess(text):
    return text
text_pipe_logit = make_pipeline(
    sktext.CountVectorizer(preprocessor=preprocess, analyzer='char', ngram_range=(1, 4)),
    LogisticRegression(n_jobs=-1, random_state=42)
)

In [28]:
label  = "Vanilla LogReg w/ CountVectorizer on char n-grams 1-4 over lemmas"
cross_validate(label, text_pipe_logit, X_lemmas, y_train)

0.9073147868300669 <- [0.91089109 0.8960396  0.90049751 0.91959799 0.90954774]


array([[863,   3,   7],
       [ 31,   8,   4],
       [ 47,   1,  39]])

Показник точності (accuracy) зріс у порівнянні з початковим, але при цьому середньоквадратичне відхилленя становить 0.008, що означає більш широкий інтервал для оцінки точності у порівнянні з 0.003 (0.907315 ± 0.008274 vs 0.899319 ± 0.003163).
Ну що ж, тоді перевіримо обидві моделі на відкладенній виборці

In [34]:
label  = "Vanilla LogReg w/ CountVectorizer on char n-grams 1-4 over lemmas"
cross_validate(label, text_pipe_logit, X_test.apply(words2lemmas), y_test)

0.8747505786575145 <- [0.86764706 0.85074627 0.86567164 0.88059701 0.90909091]


array([[286,   1,   4],
       [ 11,   2,   2],
       [ 24,   0,   5]])

In [35]:
def preprocess(text):
    return text
text_pipe_logit = make_pipeline(
    sktext.CountVectorizer(preprocessor=preprocess, analyzer='char', ngram_range=(1, 4)),
    LogisticRegression(n_jobs=-1, random_state=42)
)

In [36]:
label  = "Vanilla LogReg w/ CountVectorizer on char n-grams 1-4"
cross_validate(label, text_pipe_logit, X_test, y_test)

0.8747053502540771 <- [0.86764706 0.86567164 0.85074627 0.89552239 0.89393939]


array([[284,   3,   4],
       [ 13,   2,   0],
       [ 22,   0,   7]])

В данному випадку модель з леммами показала кращий результат. Можливо, це результат підгонки і недбалого використання тестового сету, що не дозволяє з певністю говорити про покращення якості з використанням лемм. Словник сентиментів не допоміг у покращенні моделі, гіпотетичні причини:
- розподіл слів і значень у анатованих матеріалах (наприклад, "бомба" має низьку оцінку, але в даному контексті було використано у позитивному сенсі)
- низький % перетину множин слів з словнику і у відгуках
- мала кількіть зібраних вигуків / необхідність додаткової попередньої обробки / помилка у коді
- неправильна детекція мови призвела до помилки у даних, що призвело до погіршення якості при використанні україномовного словника
- помилки у словах користувачів призвели до неможливості співставити їх з початковим словом
- неправильний поділ на позитивні/негативні/нейтральні відгуки і необхідність подальшого розбиття на речення з відповідними забарвленнями

Можливі заходи для уникнення проблем:
- ручні анотації сентиментів кожного окремого речення із відгуків
- виправлення помилок користувачів / пошук у словниках найближчих слів за редакторською відстанню
- пошук додаткових данних
- побудова векторних представлень на корпусі інтернет-відгуків / інтернет комунікацій з подібним мовним розподілом і розширення словника тональності за рахунок їх

Забув спробувати використовувати набір стоп-слів, але, враховуючи, що найкращі модель н-грамні, їх використання не мусить значно покращити результат. Подивимось, що буде з моделями на словах

In [21]:
# http://meta-ukraine.com/ua/pages/stopwrd.asp
STOP_WORDS = "навіть для де до дещо це адже авжеж що майже так такий також те тобто ледве тощо тож під отже отож як який".split(" ")

In [19]:
def preprocess(text):
    return text
text_pipe_logit = make_pipeline(
    sktext.CountVectorizer(preprocessor=preprocess, analyzer='word'),
    LogisticRegression(n_jobs=-1, random_state=42)
)
label  = "Vanilla LogReg w/ CountVectorizer on words without stop words"
cross_validate(label, text_pipe_logit, X_test, y_test)


0.8806754995078085 <- [0.86764706 0.86567164 0.88059701 0.89552239 0.89393939]


array([[290,   0,   1],
       [ 13,   2,   0],
       [ 26,   0,   3]])

In [20]:
def preprocess(text):
    return text
text_pipe_logit = make_pipeline(
    sktext.CountVectorizer(preprocessor=preprocess, analyzer='word', stop_words=STOP_WORDS),
    LogisticRegression(n_jobs=-1, random_state=42)
)
label  = "Vanilla LogReg w/ CountVectorizer on words with stop words"
cross_validate(label, text_pipe_logit, X_test, y_test)


0.8806754995078085 <- [0.86764706 0.88059701 0.86567164 0.89552239 0.89393939]


array([[291,   0,   0],
       [ 13,   2,   0],
       [ 27,   0,   2]])

точність не змінилася (додатково 1 правильно класифікований позитивний відгук і 1 неправильно класифікований негативний)